In [1]:
!pip install transformers[torch] accelerate
!pip install datasets
!pip install sentence_transformers
!pip install numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
import pandas as pd
import numpy as np
from numba import cuda
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from tqdm import tqdm
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')
df1=pd.read_csv(r'/content/drive/MyDrive/Trailadata_preprocessed.csv')

df2=pd.read_csv(r'/content/drive/MyDrive/preprocessedexp_dataset.csv')

Mounted at /content/drive


In [4]:
df1.drop(columns=['File Name','Transcript','tokenized_Transcript'],inplace=True)
df1['Type'] = 'A'

In [5]:
df2['Type'] = 'B'
df2.drop(columns=['File Name','Transcript','tokenized_Transcript'],inplace=True)

In [6]:
combined_df = pd.concat([df1, df2], ignore_index=True)

In [7]:
combined_df.head()

,Label,clean_text,Type,Unnamed: 0
0,truthful,represent people year system letter prisioner ...,A,NaN
1,truthful,reform criminal justice system matter law legi...,A,NaN
2,truthful,tell say accident say accident ian fine accide...,A,NaN
3,truthful,uh office get uh maybe close o clock remember ...,A,NaN
4,truthful,yes yep stay uh yep prob yes yes,A,NaN


In [8]:
combined_df.drop(columns=['Unnamed: 0'],inplace=True)

In [9]:
import spacy
from collections import Counter

!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
# Load Spacy's English language model
nlp = spacy.load("en_core_web_lg")

# Define a function to tokenize text
def tokenize(txt):
    # Remove punctuation and double space, and convert to lowercase:
    tokens = [token.text.lower() for token in txt if not (token.is_punct or token.is_space)]
    # Join tokens back into a string
    tokens = " ".join(tokens)
    return tokens

In [11]:
# Apply the preprocessing pipeline using nlp.pipe
tokenized_txt = []

for txt in nlp.pipe(combined_df['clean_text']):
    tokenized_txt.append(tokenize(txt))

# Add the preprocessed text as a new column in the dataframe
combined_df['tokenized_Transcript'] = tokenized_txt

combined_df.drop(columns=['clean_text'],inplace=True)

In [12]:
combined_df['Label'] = combined_df['Label'].map({'truthful': 1, 'decpetive': 0})
combined_df['Label'].fillna(0, inplace=True)
combined_df['Label'] = combined_df['Label'].astype(int)

In [13]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
# Preprocess function to tokenize input text and format labels
def preprocess_function(examples):
    inputs = examples["tokenized_Transcript"]
    targets = examples["Label"]

    # Convert labels to lists
    targets = [[label] for label in targets]

    # Ensure that inputs are lists of strings
    if isinstance(inputs, str):
        inputs = [inputs]

    # Tokenize inputs
    inputs_tokenized = tokenizer(inputs, padding=True, truncation=True)['input_ids']

    return {'input_ids': inputs_tokenized, 'labels': targets}

In [14]:
def create_dataset(df1, seed=42, cv=10):
    train = {f'split_{i+1}': [] for i in range(cv)}
    test = {f'split_{i+1}': [] for i in range(cv)}

    np.random.seed(seed)

    for k in train.keys():
        train[k] = combined_df.sample(frac=0.8, random_state=seed)
        test[k] = combined_df.drop(train[k].index)

    return train, test

In [15]:
seed = 42
numcv = 10
num_epochs = 3
model_size = 'base'

In [16]:
#!pip install --upgrade transformers accelerate
import accelerate
from sklearn.metrics import accuracy_score
train, test = create_dataset(combined_df, seed=seed, cv=numcv)
results = {}
collect_result = {}
ground_truth = {}
# Define the device
device = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
from transformers import AutoModelForSeq2SeqLM
for sp in tqdm(train.keys()):
    data_train = Dataset.from_pandas(train[sp])
    data_test = Dataset.from_pandas(test[sp])
    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
    model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
    # Tokenize and preprocess data
    data_train = data_train.map(preprocess_function, batched=True)
    data_test = data_test.map(preprocess_function, batched=True)

    training_args = Seq2SeqTrainingArguments(
        output_dir="./results",  # The output directory
        logging_dir="./logs",
        evaluation_strategy="epoch",  # Evaluation is done at the end of each epoch
        learning_rate=5e-4,
        per_device_train_batch_size=2,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=num_epochs,
        predict_with_generate=True,
        fp16=False,
        push_to_hub=False,
    )
    # Define trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=data_train,
        eval_dataset=data_test,
        tokenizer=tokenizer,
    )

    # Train the model
    trainer.train()
    # Get predictions
    predictions = trainer.predict(data_test)
    pred_labels = np.argmax(predictions.predictions, axis=1)

    # Store predictions in the original DataFrame
    combined_df.loc[test[sp].index, 'Prediction'] = pred_labels
    combined_df['Prediction'].fillna(0, inplace=True)

    # Clean up
    del model
    torch.cuda.empty_cache()



  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

Map:   0%|          | 0/229 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.973036
2,2.931400,1.043278
3,1.182900,1.124193


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


100%|██████████| 10/10 [52:57<00:00, 317.73s/it]


In [18]:
# Display final DataFrame with predictions

print("Final DataFrame with Predictions:")
print(combined_df)

# Assuming ground_truth is defined outside the loop
# Calculate accuracy
ground_truth = combined_df.loc[test[sp].index,'Label']
test_accuracy = accuracy_score(ground_truth, combined_df.loc[test[sp].index, 'Prediction'])
print("Test Accuracy:", test_accuracy)

Final DataFrame with Predictions:
      Label Type                               tokenized_Transcript  \
0         1    A  represent people year system letter prisioner ...   
1         1    A  reform criminal justice system matter law legi...   
2         1    A  tell say accident say accident ian fine accide...   
3         1    A  uh office get uh maybe close o clock remember ...   
4         1    A                   yes yep stay uh yep prob yes yes   
...     ...  ...                                                ...   
1142      0    B  travel experience visit dark hedge northern ir...   
1143      0    B  travel experience visit muchu picchu peru actu...   
1144      0    B  travel experience visit hallstatt austria actu...   
1145      0    B  trip visit jiuzhaigou valley china actually na...   
1146      0    B  trip visit bagan myanmar actually old city bud...   

      Prediction  
0            0.0  
1            0.0  
2            0.0  
3            0.0  
4            0.0  

In [19]:
file_path = 'CombinedL.csv'
combined_df.to_csv(file_path, index=False)